In [1]:
import os
os.environ["CUDA_DEVICE_OREDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dataset
#from torchsummary import summary
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import random
print(torch.__version__, torchvision.__version__)
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)

1.8.0 0.2.2
GeForce RTX 3080
11.1


In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
    
max_epochs = 30

In [3]:
# randomness 제어 
# https://hoya012.github.io/blog/reproducible_pytorch/
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)

In [4]:
# vgg16_bn pretrained false
vgg16 = torchvision.models.vgg16_bn(pretrained=False).to(device)
#summary(vgg16, input_size=(224,224,3))

In [5]:
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [10]:
# dataset load

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
dataset_path = "/media/1/Imagenet_dup/"
# imagenet data load
train_dataset = dataset.ImageFolder(root=dataset_path+"train",
                                       transform=transform)
val_dataset = dataset.ImageFolder(root=dataset_path+"val",
                                       transform=transform)
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                        batch_size=16,
                                        shuffle=False,
                                        num_workers=0)
val_dataloader = torch.utils.data.DataLoader(val_dataset,
                                        batch_size=64,
                                        shuffle=False,
                                        num_workers=0)
print(len(train_dataloader),len(val_dataloader))

80073 782


In [11]:
len(train_dataloader),len(val_dataloader)

(80073, 782)

In [ ]:
# training phase
criterion = torch.nn.CrossEntropyLoss(size_average = None).to(device)
optimizer = torch.optim.SGD(vgg16.parameters(),lr=0.0005)
lr_sche = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

vgg16.to(device)
for epoch in range(max_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader):
        inputs,labels = data
        inputs,labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        y_pred = vgg16(inputs)
        # compute loss 
        loss = criterion(y_pred,labels)
        #print(epoch,i, loss.item())
        
        if not torch.isfinite(loss):
            print("WARNING: non-finite loss, ending training")
            exit(1)
        else :
            running_loss += loss.item()
        loss.backward()
        optimizer.step()
        if i % 100 == 99: 
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/100)) 
            running_loss = 0.0

In [19]:
# evaluation phase
criterion = torch.nn.CrossEntropyLoss(size_average = None).to(device)

vgg16.cuda()
vgg16.eval()
total = 0
correct =0
with torch.no_grad():
    print("eval start")
    for i, data in enumerate(val_dataloader):
        inputs,labels = data
        inputs,labels = inputs.to(device), labels.to(device)
        
        y_hat = vgg16(inputs)
        _, predicted = torch.max(y_hat.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if(i%50 == 49):
            print("step : {} / {}".format(i + 1, int(len(val_dataset)/labels.size(0))))
            #print(".",end="")
    print("")
print("acc of vgg16 on imagenet : %.4f %%" %(100*correct/total))    

eval start
step : 50 / 781
step : 100 / 781
step : 150 / 781
step : 200 / 781
step : 250 / 781
step : 300 / 781
step : 350 / 781
step : 400 / 781
step : 450 / 781
step : 500 / 781
step : 550 / 781
step : 600 / 781
step : 650 / 781
step : 700 / 781
step : 750 / 781

acc of vgg16 on imagenet : 0.1000 %


In [20]:
vgg16 = torchvision.models.vgg16_bn(pretrained=False).to(device)
# evaluation phase
criterion = torch.nn.CrossEntropyLoss(size_average = None).to(device)

vgg16.cuda()
vgg16.eval()
total = 0
correct =0
with torch.no_grad():
    print("eval start")
    for i, data in enumerate(val_dataloader):
        inputs,labels = data
        inputs,labels = inputs.to(device), labels.to(device)
        
        y_hat = vgg16(inputs)
        _, predicted = torch.max(y_hat.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if(i%50 == 49):
            print("step : {} / {}".format(i + 1, int(len(val_dataset)/labels.size(0))))
            #print(".",end="")
    print("")
print("acc of vgg16 on imagenet : %.4f %%" %(100*correct/total))    

eval start
step : 50 / 781
step : 100 / 781
step : 150 / 781
step : 200 / 781
step : 250 / 781
step : 300 / 781
step : 350 / 781
step : 400 / 781
step : 450 / 781
step : 500 / 781
step : 550 / 781
step : 600 / 781
step : 650 / 781
step : 700 / 781
step : 750 / 781

acc of vgg16 on imagenet : 0.0940 %
